# ⚙️ Kafka Producer

#### I. Importing libraries

In [32]:
from kafka import KafkaProducer
from json import dumps

import pandas as pd

In [34]:
producer = KafkaProducer(bootstrap_servers=['54.198.23.227:9092'], value_serializer=lambda x: dumps(x).encode("utf-8"))

KeyboardInterrupt: 

#### II. Importing dataset

In [20]:
df = pd.read_csv("./data/player_stats.csv")

df.head()

,id,match_id,game_id,team,score_team,opponent,score_opp,win_lose,map,map_pick,...,rating,acs,kill,death,assist,kast%,adr,hs%,fk,fd
0,0,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,...,1.19,239,15,11,6,72%,146,16%,1,0
1,1,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,...,0.84,234,15,14,6,61%,145,32%,1,4
2,2,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,...,0.68,173,10,15,2,67%,123,18%,4,2
3,3,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,...,0.63,142,8,15,4,83%,92,29%,3,2
4,4,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,...,0.57,114,7,16,8,61%,63,13%,0,1


Adding the K/D (Kill/Death ratio) column

In [21]:
df['kd'] = (df.kill / df.death).astype(float).round(2)

Droping the Id column

In [22]:
df = df.drop(columns="id")

Renaming column with special characters

In [23]:
df = df.rename(columns={"kast%": "kast", "hs%": "hs"})

In [24]:
df.head()

,match_id,game_id,team,score_team,opponent,score_opp,win_lose,map,map_pick,player_id,...,acs,kill,death,assist,kast,adr,hs,fk,fd,kd
0,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,999,...,239,15,11,6,72%,146,16%,1,0,1.36
1,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,1395,...,234,15,14,6,61%,145,32%,1,4,1.07
2,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,424,...,173,10,15,2,67%,123,18%,4,2,0.67
3,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,1000,...,142,8,15,4,83%,92,29%,3,2,0.53
4,53658,119512,ZETA,5,DRX,13,opponent win,Ascent,opponent pick,6668,...,114,7,16,8,61%,63,13%,0,1,0.44


Example of a record to stream via the Producer

In [25]:
df.take([10]).to_dict(orient='records')[0]

{'match_id': 53658,
 'game_id': 119513,
 'team': 'ZETA',
 'score_team': 5,
 'opponent': 'DRX',
 'score_opp': 13,
 'win_lose': 'opponent win',
 'map': 'Pearl',
 'map_pick': 'team pick',
 'player_id': 6668,
 'player': 'SugarZ3ro',
 'agent': 'viper',
 'rating': '1.03',
 'acs': '229',
 'kill': 14,
 'death': 13,
 'assist': 4,
 'kast': '72%',
 'adr': '152',
 'hs': '27%',
 'fk': '1',
 'fd': '2',
 'kd': 1.08}

#### III. Simulating the Real Time data to the Consumer

In [31]:
i = 0
while i <= 500:
    game = df.take([i]).to_dict(orient='records')[0]
    producer.send('valorant_prjct', value=game)
    i+=1

In [30]:
producer.flush()